In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow_model_optimization as tfmot
import tensorflow_addons as tfa

C:\Users\Umer\AppData\Roaming\Python\Python39\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [37]:
inputPath = r"D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



for i in filesArray:
    print(i)
    
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    
    data = data/2 #Normalise

    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1]
    else:
        label = [0]
    labels.append(label)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

print(df_array.shape)
print(labels_array.shape)

['False' 'True']
2
1914
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_191958.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192022.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192026.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192043.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192126.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192132.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192140.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\New Data1\False\data_20230510_192153.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\

In [10]:
import datetime

# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def lr_schedule(epoch, lr):
    if epoch < 15:
        return lr
    else:
        print(0.001 * tf.math.exp(-0.02*((epoch-15))))
        return 0.001 * tf.math.exp(-0.02*((epoch-15)))

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    
total_epochs = 0

model = Sequential()

model.add(L.Input(shape=(1000, 1)))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=1, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=2, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=4, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=8, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=16, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=32, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=64, activation='relu'))
model.add(L.Conv1D(64, kernel_size=5, dilation_rate=100, activation='relu'))
model.add(L.GlobalAveragePooling1D())
model.add(L.Flatten())
model.add(L.Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(L.Dense(256, activation='relu'))  # Additional dense layer
model.add(L.Dropout(0.2))  # Dropout layer to reduce overfitting
model.add(L.Dense(1, activation='sigmoid'))


model.compile(optimizer=tf.optimizers.SGD(learning_rate = 0.001, momentum=0.9),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])

# model.build((235, 4001, 1))
print(model.summary())

new_epochs = 145
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 1, callbacks=[tensorboard_callback,lr_scheduler])
total_epochs += new_epochs

model.evaluate(df_array, labels_array, batch_size=1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_48 (Conv1D)          (None, 996, 64)           384       
                                                                 
 conv1d_49 (Conv1D)          (None, 988, 64)           20544     
                                                                 
 conv1d_50 (Conv1D)          (None, 972, 64)           20544     
                                                                 
 conv1d_51 (Conv1D)          (None, 940, 64)           20544     
                                                                 
 conv1d_52 (Conv1D)          (None, 876, 64)           20544     
                                                                 
 conv1d_53 (Conv1D)          (None, 748, 64)           20544     
                                                                 
 conv1d_54 (Conv1D)          (None, 492, 64)          

[0.0024012718349695206, 0.9972375631332397]

In [151]:
# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.optimizers.Adam(learning_rate=   0.00005),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])

new_epochs = 10
model.fit(df_array, labels_array, shuffle=1,  initial_epoch=total_epochs, epochs=total_epochs+new_epochs, validation_split=0.05, batch_size = 1, callbacks=[tensorboard_callback])
total_epochs += new_epochs

Epoch 187/196
1818/1818 [==============================] - 37s 20ms/step - loss: 0.0089 - binary_accuracy: 0.9868 - val_loss: 0.0202 - val_binary_accuracy: 0.9688
Epoch 188/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0063 - binary_accuracy: 0.9917 - val_loss: 0.0301 - val_binary_accuracy: 0.9792
Epoch 189/196
1818/1818 [==============================] - 38s 21ms/step - loss: 0.0047 - binary_accuracy: 0.9945 - val_loss: 0.0260 - val_binary_accuracy: 0.9896
Epoch 190/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0066 - binary_accuracy: 0.9917 - val_loss: 0.0381 - val_binary_accuracy: 0.9792
Epoch 191/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0049 - binary_accuracy: 0.9934 - val_loss: 0.0236 - val_binary_accuracy: 0.9792
Epoch 192/196
1818/1818 [==============================] - 35s 19ms/step - loss: 0.0056 - binary_accuracy: 0.9934 - val_loss: 0.0219 - val_binary_accuracy: 0.9792
Epoch 193/196
1818/181

In [110]:
total_epochs = 126

In [ ]:
model.evaluate(df_array, labels_array, batch_size=1)

In [14]:
tf.keras.models.save_model(model, 'model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [17]:

# Load your Keras model
model = tf.keras.models.load_model('model')

# Define the pruning parameters
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1,
    final_sparsity=0.99,
    begin_step=0,
    end_step=19
)

# Define the pruning callback
pruning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp')
]

# Create a pruned model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Train the pruned model
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.000001),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])
pruned_model.fit(df_array, labels_array, shuffle=1, epochs=20, validation_split=0.25, callbacks=pruning_callbacks)

# Remove the pruning wrappers from the model
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the pruned model
pruned_model.save('pruned_model',  save_format='tf')
pruned_model.save('pruned_model.h5')

Epoch 1/20
43/43 [==============================] - 16s 283ms/step - loss: 0.0030 - binary_accuracy: 0.9948 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 2/20
43/43 [==============================] - 12s 274ms/step - loss: 0.0040 - binary_accuracy: 0.9956 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 3/20
43/43 [==============================] - 12s 275ms/step - loss: 0.0029 - binary_accuracy: 0.9971 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 4/20
43/43 [==============================] - 12s 277ms/step - loss: 0.0037 - binary_accuracy: 0.9963 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 5/20
43/43 [==============================] - 12s 275ms/step - loss: 0.0036 - binary_accuracy: 0.9963 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 6/20
43/43 [==============================] - 12s 276ms/step - loss: 0.0035 - binary_accuracy: 0.9956 - val_loss: 0.0021 - val_binary_accuracy: 0.9978
Epoch 7/20
43/43 [==============================] - 12s 27

INFO:tensorflow:Assets written to: pruned_model\assets


INFO:tensorflow:Assets written to: pruned_model\assets


In [39]:
# Load your Keras model
pruned_model = tf.keras.models.load_model('pruned_model')
pruned_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.BinaryFocalCrossentropy(),
              metrics=[metrics.BinaryAccuracy()])
pruned_model.evaluate(df_array, labels_array, batch_size=1)

1914/1914 [==============================] - 6s 3ms/step - loss: 0.0030 - binary_accuracy: 0.9969


[0.0030205186922103167, 0.9968652129173279]

In [19]:
converter = tf.lite.TFLiteConverter.from_saved_model("pruned_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


with open('model_quantized.tflite', 'wb') as f:
  f.write(tflite_model)

In [20]:
interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
interpreter.allocate_tensors()

In [21]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

In [23]:
print(input_details[0]['shape'])
print(df_array.shape)
old_df = df_array

[   1 1000    1]
(1810, 1000)


In [24]:

df_array = np.expand_dims(old_df, axis=-1) # add extra dimension to the end

In [25]:
correct = 0
total = 0
from scipy.stats import pearsonr


for i in range(len(df_array)):
    interpreter.set_tensor(input_details[0]['index'], df_array[i:i+1])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    origional = model.predict(df_array[i:i+1], verbose=0)
        
    print(f'Label: {labels_array[i]}    tf model prediction: {origional}    tflite model prediction: {output}')
    prediction = np.argmax(output)
    mape = abs(origional - output)
    print(mape)
    if(mape < 0.01):
            correct += 1
    
    total += 1
    print(f'{correct} of {total} predictions were accurate to within 1%')

accuracy = correct / total
print('Accuracy:', accuracy)

Label: [0]    tf model prediction: [[2.8769675e-06]]    tflite model prediction: [[2.785277e-06]]
[[9.169048e-08]]
1 of 1 predictions were accurate to within 1%
Label: [1]    tf model prediction: [[0.99660766]]    tflite model prediction: [[0.9965538]]
[[5.38826e-05]]
2 of 2 predictions were accurate to within 1%
Label: [0]    tf model prediction: [[2.8643058e-06]]    tflite model prediction: [[2.7943336e-06]]
[[6.99722e-08]]
3 of 3 predictions were accurate to within 1%
Label: [1]    tf model prediction: [[0.99996877]]    tflite model prediction: [[0.99996716]]
[[1.6093254e-06]]
4 of 4 predictions were accurate to within 1%
Label: [0]    tf model prediction: [[2.8617583e-06]]    tflite model prediction: [[2.7930814e-06]]
[[6.8676854e-08]]
5 of 5 predictions were accurate to within 1%
Label: [0]    tf model prediction: [[2.8621405e-06]]    tflite model prediction: [[2.7949518e-06]]
[[6.7188694e-08]]
6 of 6 predictions were accurate to within 1%
Label: [0]    tf model prediction: [[0.03